In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext()

In [2]:
from operator import add

In [3]:
inputfile_RDD = sc.textFile("./cause-of-death-by-age-group.csv")

In [4]:
# remove icd_3byte because it causes parsing errors
# year,icd_main,death_age,gender,count
cleaned_inputfile_RDD = inputfile_RDD.map(lambda line: line.split(","))\
                                        .map(lambda data: (int(data[0]), \
                                                           data[1].split("-")[2].split("(")[0], \
                                                           data[len(data)-3].split(") ")[1], \
                                                           data[len(data)-2], \
                                                           int(data[len(data)-1])))
cleaned_inputfile_RDD.collect()

[(2014, 'Certain infection and parasitic diseases', 'Under 28 days', 'F', 0),
 (2014,
  'Certain infection and parasitic diseases',
  '28 days - under 1 years',
  'F',
  0),
 (2014, 'Certain infection and parasitic diseases', '1', 'F', 0),
 (2014, 'Certain infection and parasitic diseases', '2', 'F', 0),
 (2014, 'Certain infection and parasitic diseases', '3', 'F', 0),
 (2014, 'Certain infection and parasitic diseases', '4', 'F', 0),
 (2014, 'Certain infection and parasitic diseases', '5 - 9', 'F', 0),
 (2014, 'Certain infection and parasitic diseases', '10 - 19', 'F', 0),
 (2014, 'Certain infection and parasitic diseases', '20 - 29', 'F', 0),
 (2014, 'Certain infection and parasitic diseases', '30 - 39', 'F', 0),
 (2014, 'Certain infection and parasitic diseases', '40 - 49', 'F', 0),
 (2014, 'Certain infection and parasitic diseases', '50 - 59', 'F', 0),
 (2014, 'Certain infection and parasitic diseases', '60 - 64', 'F', 0),
 (2014, 'Certain infection and parasitic diseases', '65 - 69

In [5]:
def age_to_range(age):
    if (age > 84):
        return '85 and Over'
    elif (age > 79):
        return '80 - 84'
    elif(age > 74):
        return '75 - 79'
    elif(age > 69):
        return '70 - 74'
    elif (age > 64):
        return '65 - 69'
    elif (age > 59):
        return '60 - 64'
    elif (age > 49):
        return '50 - 59'
    elif (age > 39):
        return '40 - 49'
    elif (age > 29):
        return '30 - 39'
    elif (age > 19):
        return '20 - 29'
    elif (age > 9):
        return '10 - 19'
    elif (age > 4):
        return '5 - 9'
    else: return str(age)

In [6]:
def value(key_value_pair):
    return key_value_pair[1]

In [7]:
gender = 'M'
age = 20

In [8]:
causes = cleaned_inputfile_RDD.filter(lambda data : data[3] == gender)\
                                .filter(lambda data : data[2] == age_to_range(age))\
                                .map(lambda data: (data[1], data[4]))\
                                .reduceByKey(add)\
                                .collect()

In [9]:
sorted(causes, key=value, reverse=True)

[('External causes of morbidity and mortality', 395),
 ('Neplasms', 63),
 ('Diseases of the circulatory system', 63),
 ('Diseases of the respiratory system', 46),
 ('Diseases of the nervous system', 14),
 ('Symptoms', 12),
 ('Certain infection and parasitic diseases', 9),
 ('Congenital malformations', 7),
 ('Diseases of the digestive system', 7),
 ('Endocrine', 4),
 ('Diseases of the genitourinary system', 3),
 ('Diseases of the musculoskeletal system and connective tissue', 1),
 ('Diseases of the skin and subcutaneous tissue', 0),
 ('Pregnancy', 0),
 ('Diseases of the ear and mastoid process', 0),
 ('Mental and behavioural disorders', 0),
 ('Diseases of the blood and blood', 0),
 ('Diseases of the eye and adnexa', 0),
 ('Certain conditions originating in the perinatal period', 0)]